# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.readthedocs.io/) and [Doxygen documentation](https://codedocs.xyz/CEED/libCEED/).

## Setting up libCEED for Python

Clone or download libCEED by running

In [ ]:
! git clone https://github.com/CEED/libCEED.git

Then move to the libCEED folder

In [ ]:
cd libCEED

And compile the base library by running

In [ ]:
! make

Build libCEED for Python by running

In [ ]:
python setup.py build install

Or without superuser permissions

In [ ]:
python setup.py build install --user

## CeedVector

Here we show some basic examples to illustrate the `CeedVector` class. In libCEED, CeedVectors constitute the main data structure and serve as input/output for `CeedOperator`. 

We illustrate the simple creation of a `CeedVector`, how to specify its size, and how to read or manipulate its data.

In [1]:
import libceed

ceed = libceed.Ceed()

n = 10
x = ceed.Vector(n)

You can visualize the (unset) CeedVector by typing

In [2]:
print(x)

The size of the `CeedVector` can also be specified as 

In [3]:
x = ceed.Vector(size=10)

* In the following example, we associate the data stored in a `CeedVector` with a `numpy.array` and use it to set and read the CeedVector's data

In [4]:
import numpy as np

ceed = libceed.Ceed()
x = ceed.Vector(size=3)

a = np.arange(1, 4, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)
b = x.get_array_read()
x.restore_array_read()

print(b)

[1. 2. 3.]


* In the following example, we set all entries in a `CeedVector` to the same value

In [5]:
ceed = libceed.Ceed()
x = ceed.Vector(size=5)

x.set_value(10)

print(x)

CeedVector length 5
  10.000000
  10.000000
  10.000000
  10.000000
  10.000000



* In the following example, we set one vector from the array of another vector

In [6]:
ceed = libceed.Ceed()
n = 10

x = ceed.Vector(n)
y = ceed.Vector(n)

a = np.arange(1, 1 + n, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

x_array = x.get_array()
y.set_array(x_array, cmode=libceed.USE_POINTER)
x.restore_array()

y_array = y.get_array_read()
print(y_array)
y.restore_array_read()

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


* In the following example, we access and modify only one entry of the `CeedVector`

In [7]:
n = 10

x = ceed.Vector(n)
a = np.zeros(n, dtype="float64")
x.set_array(a, cmode=libceed.USE_POINTER)

b = x.get_array()
b[3] = -3.14;
x.restore_array()
a[3]

-3.14

* In the following example, we compute the $L_1$, $L_2$ (default), and $L_{\infty}$ norms of a `CeedVector` (keeping in mind that these are local norm computations; not accurate for parallel executions with no reductions).

In [8]:
n = 10
x = ceed.Vector(n)

a = np.arange(0, n, dtype="float64")
for i in range(n):
  if (i % 2 == 0):
    a[i] *= -1
x.set_array(a, cmode=libceed.USE_POINTER)

norm_1 = x.norm(normtype=libceed.NORM_1)
print(norm_1)

norm_2 = x.norm()
print(norm_2)

norm_max = x.norm(normtype=libceed.NORM_MAX)
print(norm_max)

45.0
16.881943016134134
9.0
